In [ ]:
#e.g.bin_width = 0.001
#session = cache.get_session_data(sessions.index.values[0])


def xcorr_lag(session, rec_area, bin_width, max_lag):
    
    session_units = session.units
    
    rec_area_1 = rec_area
    rec_units_1 = session_units[session_units.structure_acronym == rec_area_1]
    rec_units_1 = rec_units_1.sort_values(by=['probe_vertical_position'], ascending=False)
    
    stim_table = session.get_presentations_for_stimulus(stimulus_type)
    stim_types = len(stim_table.unique())
    
    num_stims = len(stim_types)
    num_units = len(sample_units)
    
    peak_matrix = np.zeros((num_units, num_units, num_stims)) 
    lag_matrix = np.zeros((num_units, num_units, num_stims))
    
    bins = np.arange(0,0.5,bin_width)

    for trials in range(num_stims):    

        stim_presentation_ids = stimulus_ids

        histograms = session.presentationwise_spike_counts(
            bin_edges=bins,
            stimulus_presentation_ids=stim_presentation_ids,
            unit_ids=units.index.values
        )

        mean_histograms = histograms.mean(dim="stimulus_presentation_id")
        rates = mean_histograms/bin_width

        for i in range(num_units):

            for j in range(num_units):

                p = plt.xcorr(rates[:,i], rates[:,j], usevlines=True, maxlags=maxlags, normed=True, lw=2)

                corr_df = pd.DataFrame(p[1], p[0], columns = ["corr_R"])

                peak_corr = corr_df.sort_values(by = ["corr_R"], ascending = False)

                peak_lag = (peak_corr.corr_R[peak_corr.index.values[0]])

                if peak_corr.index[0] > 0:

                    peak_matrix[i, j, trials] = peak_lag

                elif peak_corr.index[0] == 0:
                    peak_matrix[i, j, trials] = np.inf

                elif peak_corr.index[0] < 0: 

                    peak_matrix[i, j, trials] = -(peak_lag)

                peak_lag = peak_corr.index.values[0]

                lag_matrix[i, j, trials] = peak_lag 

    return peak_matrix, lag_matrix


## E.g. can plot with 

#fig,ax = plt.subplots(4, 2, figsize=(8,16))
#ax = ax.ravel()

#for i, a in enumerate(ax):
#    a.imshow(peak_matrix[:,:,i], aspect="auto", cmap=plt.get_cmap('bwr'))
